# TensorFlow实现skip-gram
原文作者天雨粟，文章来源：
[一文详解 Word2vec 之 Skip-Gram 模型（实现篇）](https://www.leiphone.com/news/201706/QprrvzsrZCl4S2lw.html)
<br/>
文章主要有以下四部分构成：
* 数据预处理
* 训练样本构建
* 模型构建
* 模型验证

In [1]:
import time
import numpy as np
import tensorflow as tf
import random
from collections import Counter

## 加载数据

In [2]:
with open('text8') as f:
    text = f.read()

## 数据预处理
预处理过程主要包括：
* 替换文本中特殊符号并去除低频词
* 对文本分词
* 构建语料
* 单词映射表

In [3]:
def preprocess(text, freq=5):
    '''
    对文本进行预处理
    text: 文本数据
    freq: 词频阈值
    '''
    # 对文本中符号进行替换
    text = text.lower()
    text = text.replace('.',' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # 删除低频词汇，减少噪声影响
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > freq]
    
    return trimmed_words

In [4]:
# 清洗文本并分词
words = preprocess(text)
print(words[:50])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


In [5]:
# 构建映射表
vocab = set(words)
vocab_to_int = {w:c for c,w in enumerate(vocab)}
int_to_vocab = {c:w for c,w in enumerate(vocab)}

In [6]:
print("total words: {}".format(len(words)))
print("total unique words {}".format(len(vocab)))

total words: 16680599
total unique words 63641


In [7]:
# 将文本转为数字
# 这里是将频次当做单词的数值表示，可能某些单词具有相同的频次从而获得
# 相同的数值表示
int_words = [vocab_to_int[w] for w in words]

In [8]:
print(int_words[:50])

[63504, 49115, 38041, 55048, 11374, 51440, 705, 11778, 56110, 25834, 63004, 45544, 59620, 59349, 13082, 32916, 18361, 51440, 32916, 195, 34650, 17408, 32916, 7616, 22565, 51440, 32916, 34842, 34650, 26248, 32916, 11374, 35781, 35783, 56110, 62923, 55048, 16536, 51233, 59877, 62669, 45236, 59470, 10109, 56110, 60059, 23767, 59877, 62204, 32916]


## 采样
严格意义上讲，这里根本不是采样。这里仅仅就是对高频词汇进行了删除。
<br/>
对停用词进行采样，例如“the”，“of”，“for”等单词进行剔除。
采用公式：
$$
P(w_i) = 1 - \sqrt{\frac{t}{f(w_i}}
\\
t: 阈值参数，一般取1e-3 至 1e-5
\\
f(w_i): 单词w_i在整个数据集中出现的频次
\\
P(w_i)：单词被删除的概率
$$

In [10]:
t = 1e-5
threshold = 0.8 # 剔除概率阈值
# 统计单词出现频次
int_word_counts = Counter(int_words) # 每个单词的对应的数量
total_count = len(int_words) # 所有单词的总数
# 计算频次
word_freqs = {w:c/total_count for w,c in int_word_counts.items()}
# 计算被删除的概率
prob_drop = {w: 1-np.sqrt(t / word_freqs[w]) for w in int_word_counts}
# 对单词进行采样
train_words  = [w for w in int_words if prob_drop[w] < threshold]

In [11]:
len(train_words)

6925252

## 构造batch
Skip-Gram模型是通过输入词来预测上下文。
<br/>
对于一个给定词，离它越近的词可能与它越相关，离它越远的词越不相关，这里我们设置窗口大小为5，对于每个训练单词，我们还会在\[1:5\]之间随机生成一个整数R，用R作为我们最终选择output word的窗口大小。这里之所以多加了一步随机数的窗口重新选择步骤，是为了能够让模型更聚焦于当前input word的邻近词。

In [27]:
def get_targets(words, idx, window_size=5):
    '''
    获得input word的上下文单词列表
    words: 单词列表
    idx: input words的索引号
    window_size: 窗口大小
    '''
    target_window = np.random.randint(1, window_size+1) # 随机生成目标窗口大小
    # 这里要考虑input word前面单词不够的情况
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    targets = set(words[start_point:idx] + words[idx+1:end_point+1])
    return list(targets)

In [13]:
def get_batches(words, batch_size, window_size = 5):
    '''
    构造一个获取batch的生成器
    '''
    # 有多少个小批量，每个批量的单词数是batch_size
    n_batches = len(words) // batch_size
    
    # 仅取full batches
    words = words[:n_batches * batch_size]
    
    for idx in range(0, len(words), batch_size):
        x,y = [], []
        batch = words[idx:idx+batch_size]
        for i in range(len(batch)):
            # input word
            batch_x = batch[i]
            # output word
            batch_y = get_targets(batch, i, window_size)
            # 由于一个input word会对应多个output word，因此需要统一长度
            x.extend([batch_x] * len(batch_y))
            y.extend(batch_y)
        yield x,y

## 构建网络
该部分主要包括：
* 输入层
* Embedding
* Negative Sampling

### 输入

In [14]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')

### Embedding
嵌入矩阵的形状为$vocab \times hidden_units_size$
<br/>
TensorFlow中的tf.nn.embedding_lookup函数可以实现lookup的计算方式

In [16]:
vocab_size = len(int_to_vocab)
embedding_size = 200 # 嵌入维度

In [17]:
with train_graph.as_default():
    # 嵌入层权重矩阵
    embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size]))
    # 实现lookup
    embed = tf.nn.embedding_lookup(embedding, inputs)

### Negative Sampling
负采样主要解决梯度下降计算速度慢问题。
<br/>
TensorFlow中的tf.nn.sampled_softmax_loss会在softmax层上进行采样计算损失，计算出的loss要比full softmax loss低

In [19]:
# 负例数量
n_sampled = 100

with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev = 0.1))
    softmax_b = tf.Variable(tf.zeros(vocab_size))
    
    # 计算negative sampling下的损失
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, vocab_size)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## 验证
为了更加直观的看到我们训练的结果，我们将查看训练出的相近语义的词。

In [21]:
with train_graph.as_default():
    # 随机挑选一些单词
    valid_size = 16
    valid_window = 100
    # 从不同位置各选8各单词
    valid_examples = np.array(random.sample(range(valid_window), valid_size // 2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000, 1000+valid_window), valid_size//2))
    
    valid_size = len(valid_examples)
    # 验证单词集
    valid_dataset = tf.constant(valid_examples, dtype = tf.int32)
    
    # 计算每个单词向量的模并进行单位化
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    # 查找验证单词的词向量
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    # 计算余弦相似度
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [30]:
epochs = 10 # 迭代次数
batch_size = 100 # batch大小
window_size = 10 # 窗口大小

with train_graph.as_default():
    saver = tf.train.Saver() # 文件存储
    
with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())
    
    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        
        for x,y in batches:
            feed = {inputs:x, labels:np.array(y)[:,None]}
            train_loss, _ = sess.run([cost,optimizer], feed_dict = feed)
            
            loss += train_loss
            
            if iteration % 100 == 0:
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
                
            # 计算相似的词
            if iteration % 1000 == 0:
                # 计算similarity
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # 取最相似的单词的前8个
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log = 'Nearest to [%s]:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
            
        save_path = saver.save(sess, "checkpoints/text8.ckpt")
        embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 3.4194 0.1995 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 2.8638 0.1862 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 2.4914 0.1872 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 2.6493 0.2007 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 2.7031 0.1949 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 2.8114 0.1935 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 2.9569 0.1808 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 3.0955 0.1818 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 3.3665 0.1802 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 3.5944 0.1796 sec/batch
Nearest to [outlived]: dl, duomo, formalization, destroyer, mug, infinitesimal, rodger, abydos,
Nearest to [rated]: psychomotor, obj, worcester, stopwatch, offroad, dragging, wavelet, hogshead,
Nearest to [dragster]: foonly, functioned, maradns, cnts, clemency, icosahedron, wold, bonds,
Nearest to 

Epoch 1/10 Iteration: 4100 Avg. Training loss: 3.9283 0.2038 sec/batch
Epoch 1/10 Iteration: 4200 Avg. Training loss: 4.2016 0.2009 sec/batch
Epoch 1/10 Iteration: 4300 Avg. Training loss: 4.6410 0.1998 sec/batch
Epoch 1/10 Iteration: 4400 Avg. Training loss: 4.5607 0.1868 sec/batch
Epoch 1/10 Iteration: 4500 Avg. Training loss: 4.0827 0.1917 sec/batch
Epoch 1/10 Iteration: 4600 Avg. Training loss: 4.4050 0.1901 sec/batch
Epoch 1/10 Iteration: 4700 Avg. Training loss: 4.2389 0.1863 sec/batch
Epoch 1/10 Iteration: 4800 Avg. Training loss: 4.5417 0.1925 sec/batch
Epoch 1/10 Iteration: 4900 Avg. Training loss: 4.7632 0.1876 sec/batch
Epoch 1/10 Iteration: 5000 Avg. Training loss: 4.3934 0.1827 sec/batch
Nearest to [outlived]: dl, duomo, mug, infinitesimal, mgb, zama, lucio, reuel,
Nearest to [rated]: obj, psychomotor, stopwatch, wavelet, whirlpool, jacks, deceptively, hogshead,
Nearest to [dragster]: foonly, functioned, maradns, wold, stylists, eclac, icosahedron, cnts,
Nearest to [simpli

Epoch 1/10 Iteration: 8100 Avg. Training loss: 3.7840 0.1942 sec/batch
Epoch 1/10 Iteration: 8200 Avg. Training loss: 3.7855 0.1838 sec/batch
Epoch 1/10 Iteration: 8300 Avg. Training loss: 3.9699 0.1835 sec/batch
Epoch 1/10 Iteration: 8400 Avg. Training loss: 4.5165 0.1829 sec/batch
Epoch 1/10 Iteration: 8500 Avg. Training loss: 4.0307 0.1827 sec/batch
Epoch 1/10 Iteration: 8600 Avg. Training loss: 3.6334 0.1873 sec/batch
Epoch 1/10 Iteration: 8700 Avg. Training loss: 3.8333 0.1914 sec/batch
Epoch 1/10 Iteration: 8800 Avg. Training loss: 4.1146 0.2031 sec/batch
Epoch 1/10 Iteration: 8900 Avg. Training loss: 4.4231 0.1819 sec/batch
Epoch 1/10 Iteration: 9000 Avg. Training loss: 4.1436 0.1822 sec/batch
Nearest to [outlived]: mug, dl, duomo, mgb, reuel, turismo, zama, tuniit,
Nearest to [rated]: obj, psychomotor, stopwatch, dragging, frown, chondrichthyes, wavelet, paternoster,
Nearest to [dragster]: foonly, maradns, wold, stylists, eclac, icosahedron, cnts, paisiello,
Nearest to [simplic

KeyboardInterrupt: 

## Visualizing the word vectors

In [31]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [32]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

NameError: name 'embed_mat' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :]), color='steelblue'
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)